In [166]:
from datasets import load_dataset

ds_book_corpus = load_dataset("bookcorpus/bookcorpus", trust_remote_code=True)

In [73]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import torch.nn.functional as F
import numpy as np

# Load the RoBERTa model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

# def calculate_perplexity(sentence, model, tokenizer):
#     encoded_input = tokenizer(sentence, return_tensors='pt', truncation=True)
#     input_ids = encoded_input['input_ids'].squeeze()  # Shape: [seq_len]
#     attention_mask = encoded_input['attention_mask']
    
#     total_log_prob = 0.0
#     N = len(input_ids)
    
#     for i in range(1, N-1):  # Skip [CLS] and [SEP] if present
#         masked_input_ids = input_ids.clone()  # Clone to avoid in-place modification
#         masked_input_ids[i] = tokenizer.mask_token_id  # Mask the i-th token
        
#         # Predict the masked token
#         with torch.no_grad():
#             outputs = model(input_ids=masked_input_ids.unsqueeze(0), attention_mask=attention_mask)
#             logits = outputs.logits
        
#         # Convert logits to probabilities
#         predicted_probs = F.softmax(logits[0, i], dim=-1)
        
#         # Get the probability of the original token
#         original_token_id = input_ids[i].item()
#         original_token_prob = predicted_probs[original_token_id].item()
        
#         # Add the log probability of the original token
#         total_log_prob += np.log(original_token_prob)
    
#     # Compute pseudo-perplexity
#     avg_log_prob = total_log_prob / (N - 2)  # Exclude [CLS] and [SEP] tokens
#     pseudo_perplexity = np.exp(-avg_log_prob)
    
#     return pseudo_perplexity

def score(sentence, model, tokenizer):
    # https://arxiv.org/abs/1910.14659
    tensor_input = tokenizer(sentence, return_tensors='pt', truncation=True)
    repeat_input = tensor_input['input_ids'].repeat(tensor_input['input_ids'].size(-1)-2, 1)
    mask = torch.ones(tensor_input['input_ids'].size(-1) - 1).diag(1)[:-2]
    masked_input = repeat_input.masked_fill(mask == 1, tokenizer.mask_token_id)
    labels = repeat_input.masked_fill( masked_input != tokenizer.mask_token_id, -100)
    with torch.inference_mode():
        loss = model(masked_input, labels=labels).loss
    return np.exp(loss.item())


# Example usage
sentence = "London is the capital of Great Britain."
pseudo_perplexity = score(sentence, model, tokenizer)
print("Pseudo-perplexity:", pseudo_perplexity)



Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Pseudo-perplexity: 1.0968683577162155


In [167]:

from transformers import AutoModelForCausalLM, AutoTokenizer
gpt_tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
gpt_model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")


# Load the RoBERTa model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("controlled/models/RoBERTa")
model = AutoModelForMaskedLM.from_pretrained("controlled/models/RoBERTa")


In [169]:

from ml_security.utils.utils import get_device
from ml_security.utils.nlp_utils import calculate_perplexity

DEVICE = get_device(allow_mps=False)

Using CPU


In [152]:
file_path = "experiment-02.txt"
file_path_ids = file_path.replace(".txt", "") + "-idxs" + ".txt"
output_file = file_path.replace(".txt", "") + "-results" + ".json"

import zlib
from tqdm import tqdm
import json

with open(file_path_ids, 'r') as f:
    lines = f.readlines()
    idxs = [int(line.strip()) for line in lines]
# repeat each entry twice, but have them in sequence
idxs = [idx for idx in idxs for _ in range(2)]


# with open(file_path, 'r') as f:
#     lines = f.readlines()
#     for line_idx, line in tqdm(enumerate(lines), total=len(lines)):
#         perpl = score(line, model, tokenizer)
#         zlib_entropy = len(zlib.compress(bytes(line, 'utf-8')))
#         gpt2_perpl = calculate_perplexity(line, gpt_model, gpt_tokenizer, DEVICE).item()
#         with open(output_file, 'a') as f:
#             f.write(json.dumps({"line": line, "perplexity": perpl, "zlib_entropy": zlib_entropy, "gpt2_perplexity": gpt2_perpl, "idx": idxs[line_idx]}) + "\n")


In [165]:
# For each id in idxs, get the corresponding text from ds_book_corpus
for idx in tqdm(idxs):
    line = ds_book_corpus['train'][idx]['text']
    tokens = tokenizer.tokenize(line)
    # get a token at random and replace it with a mask token
    mask_idx = np.random.randint(len(tokens))
    original_token = tokens[mask_idx]
    tokens[mask_idx] = tokenizer.mask_token
    sentence = " ".join(tokens)

    tokens_input = tokenizer(sentence, return_tensors='pt', truncation=True)

    
    model.eval()
    with torch.no_grad():
        outputs = model(tokens_input['input_ids'], return_dict=True)
        logits = outputs.logits
        predicted_index = torch.argmax(logits[0, mask_idx]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        print(f"Original token: {original_token}, Predicted token: {predicted_token}")
        print(f"Original sentence: {line}")
        print(f"Masked sentence: {sentence}")
        print(f"Predicted sentence: {sentence.replace(tokenizer.mask_token, predicted_token)}")
        print("\n")



  0%|          | 6/2000 [00:00<01:18, 25.54it/s]

Original token: you, Predicted token: `
Original sentence: `` you are threatening me ? ''
Masked sentence: ` ` [MASK] are threatening me ? ' '
Predicted sentence: ` ` ` are threatening me ? ' '


Original token: ', Predicted token: ?
Original sentence: `` you are threatening me ? ''
Masked sentence: ` ` you are threatening me ? [MASK] '
Predicted sentence: ` ` you are threatening me ? ? '


Original token: ##hip, Predicted token: stars
Original sentence: you can click on the image to get a bigger view : entry 2-307 : october 29 , 2014 the milk run preview : starship omcom one of the key tenets of the upcoming novel entitled the milk run was that i tied up any remaining loose ends from the previous novels in the rome 's revolution series .
Masked sentence: you can click on the image to get a bigger view : entry 2 - 307 : october 29 , 2014 the milk run preview : stars [MASK] om ##com one of the key ten ##ets of the upcoming novel entitled the milk run was that i tied up any remaining loo

  1%|          | 13/2000 [00:00<01:08, 29.14it/s]

Original token: chuckled, Predicted token: he
Original sentence: he chuckled as he folded her arm within his .
Masked sentence: he [MASK] as he folded her arm within his .
Predicted sentence: he he as he folded her arm within his .


Original token: chuckled, Predicted token: he
Original sentence: he chuckled as he folded her arm within his .
Masked sentence: he [MASK] as he folded her arm within his .
Predicted sentence: he he as he folded her arm within his .


Original token: middle, Predicted token: the
Original sentence: it 's the middle of the week . ''
Masked sentence: it ' s the [MASK] of the week . ' '
Predicted sentence: it ' s the the of the week . ' '


Original token: the, Predicted token: s
Original sentence: it 's the middle of the week . ''
Masked sentence: it ' s [MASK] middle of the week . ' '
Predicted sentence: it ' s s middle of the week . ' '


Original token: ., Predicted token: sure
Original sentence: `` i ... i 'm not sure . ''
Masked sentence: ` ` i . . . i ' 

  1%|          | 17/2000 [00:00<01:04, 30.56it/s]

...

  7%|▋         | 139/2000 [00:05<01:05, 28.44it/s]

Original token: sent, Predicted token: i
Original sentence: 'i sent some of our men round the back way to intercept those pazzi rats .
Masked sentence: ' i [MASK] some of our men round the back way to intercept those paz ##zi rats .
Predicted sentence: ' i i some of our men round the back way to intercept those paz ##zi rats .


Original token: know, Predicted token: i
Original sentence: i know he loves you .
Masked sentence: i [MASK] he loves you .
Predicted sentence: i i he loves you .


Original token: he, Predicted token: know
Original sentence: i know he loves you .
Masked sentence: i know [MASK] loves you .
Predicted sentence: i know know loves you .


Original token: away, Predicted token: looked
Original sentence: nothing , rupert looked away from the vampire .
Masked sentence: nothing , rupert looked [MASK] from the vampire .
Predicted sentence: nothing , rupert looked looked from the vampire .


Original token: the, Predicted token: from
Original sentence: nothing , rupert lo

  7%|▋         | 142/2000 [00:05<01:12, 25.62it/s]


Original token: in, Predicted token: lived
Original sentence: he lived in a small house .
Masked sentence: he lived [MASK] a small house .
Predicted sentence: he lived lived a small house .


Original token: ., Predicted token: bacteria
Original sentence: the bacteria .
Masked sentence: the bacteria [MASK]
Predicted sentence: the bacteria bacteria


Original token: the, Predicted token: .
Original sentence: the bacteria .
Masked sentence: [MASK] bacteria .
Predicted sentence: . bacteria .




KeyboardInterrupt: 

In [122]:
import pandas as pd
columns = ['perplexity', 'zlib_entropy', 'gpt2_perplexity', 'line', 'idx']
df_plots = pd.DataFrame(columns=columns)

with open(output_file, 'r') as f:
    # read line by line
    for line in f:
        # parse the line
        data = json.loads(line)
        # create a dictionary
        data_dict = {col: data[col] for col in columns}
        # create a dataframe
        df = pd.DataFrame(data_dict, index=[0])
        # append to the main dataframe
        df_plots = pd.concat([df_plots, df], ignore_index=True)




/var/folders/b4/lwfgccm95kqd2skcwvrt2fr00000gn/T/ipykernel_70482/1893091895.py:15: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [123]:
df_plots

,perplexity,zlib_entropy,gpt2_perplexity,line,idx
0,4.322009,139,66.710045,"` ` you are threatening life , especially to d...",48848192
1,3.961453,131,174.877014,` ` you are threatening her . ' ` snipers whic...,48848192
2,4.382756,132,167.183807,you can click on the stash to make it her favo...,13015892
3,4.287555,142,118.102234,"you can click on the videos or the audio , or ...",13015892
4,11.184225,139,129.840439,it was beyond all the perukers to understand t...,53532639
...,...,...,...,...,...
1995,4.185416,112,19.498312,"so for once , i was all kinds of happy with th...",63445738
1996,3.370690,141,221.380371,"through the door , i heard another one of my f...",27028465
1997,2.418173,133,50.049370,"through the door , i slowly turned around , an...",27028465
1998,9.500776,145,207.949646,"he smiled and stared at the other boys , wonde...",68518599


In [146]:
df_plots['id'] = df_plots.index
df_plots['selected'] = False
# df_plots['selected'] = (df_plots['perplexity'] < 15) & (df_plots['gpt2_perplexity'] > 35)

def calculate_3gram_accuracy(reference_text, predicted_text):
    def generate_ngrams(text, n):
        words = text.split()
        return [tuple(words[i:i+n]) for i in range(len(words)-n+1)]
    
    reference_3grams = generate_ngrams(reference_text, 3)
    predicted_3grams = generate_ngrams(predicted_text, 3)
    
    matching_3grams = set(reference_3grams) & set(predicted_3grams)
    accuracy = len(matching_3grams) / len(reference_3grams) if reference_3grams else 0
    
    return accuracy

df_plots['3gram_accuracy'] = df_plots.apply(lambda row: calculate_3gram_accuracy(row['line'], ds_book_corpus['train'][row['idx']]["text"]), axis=1)



In [147]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px


# plot scatter plot with regression line
fig = px.scatter(df_plots, x='perplexity', y='gpt2_perplexity', hover_data=['line', 'id', 'zlib_entropy'], color='selected')

# show the plot
fig.show()

In [148]:
df_plots.iloc[212]['line']

'the fight between man and machine . five films based on five major box office hit films in the great depression , / 1993 . the first attack on al dealey , / 1995 the life of barry levinson , / 1997 .\n'

In [149]:
#df_filtered = df_plots[df_plots['selected']]
df_filtered = df_plots.copy()

In [150]:
df_filtered

,perplexity,zlib_entropy,gpt2_perplexity,line,idx,id,selected,3gram_accuracy
0,4.322009,139,66.710045,"` ` you are threatening life , especially to d...",48848192,0,False,0.023256
1,3.961453,131,174.877014,` ` you are threatening her . ' ` snipers whic...,48848192,1,False,0.025000
2,4.382756,132,167.183807,you can click on the stash to make it her favo...,13015892,2,False,0.105263
3,4.287555,142,118.102234,"you can click on the videos or the audio , or ...",13015892,3,False,0.073171
4,11.184225,139,129.840439,it was beyond all the perukers to understand t...,53532639,4,False,0.071429
...,...,...,...,...,...,...,...,...
1995,4.185416,112,19.498312,"so for once , i was all kinds of happy with th...",63445738,1995,False,0.069767
1996,3.370690,141,221.380371,"through the door , i heard another one of my f...",27028465,1996,False,0.100000
1997,2.418173,133,50.049370,"through the door , i slowly turned around , an...",27028465,1997,False,0.071429
1998,9.500776,145,207.949646,"he smiled and stared at the other boys , wonde...",68518599,1998,False,0.075000


In [151]:
for row in df_filtered.iterrows():
    gram_accuracy = row[1]['3gram_accuracy']
    if gram_accuracy > 0.12:
        print(row[1]['3gram_accuracy'])
        print(row[1]['line'])
        print(ds_book_corpus['train'][row[1]['idx']]["text"])
        print()

0.125
her head lay on his chest , smelled so sweet , remembered her scent , the unexpectedly raw sensation of him kissing beneath her chemise , her amniotic fluid , and then looking at him like he was being utterly ridiculous .

her head lay on his chest , her fingertips tracing her name imprinted in his skin .

0.14634146341463414
she wanted to remain mad at him , to play a little cat and mouse game and soon they were just a weirder - than - usual pair . what was the conversation between her three seven - day roommates looked like ?

she wanted to remain mad at him , but she knew she would forgive him because it was still dark .

0.1282051282051282
he asked me to accompany him to a bar in virginia beach , where we both had cocktails , and i readily agreed . all around us were people , long - haired blonds and guys , women and bloodchemps .

he asked me to accompany him in a reel , and i accepted , thinking nothing of it .

0.12195121951219512
* * * merlin spent the next week assisting

In [141]:
# create a matrix to calculate the n_gram accuracy comparing the generated text with entries from the book corpus
smaller_ds_book_corpus = ds_book_corpus['train'].select(list(range(1000)))

output_file = file_path.replace(".txt", "") + "-results" + "-ngram" + ".json"
import json

for i, row in tqdm(df_filtered.iterrows(), total=len(df_filtered)):
    for j, book_sentence in enumerate(smaller_ds_book_corpus):
        n_gram_accuracy = calculate_3gram_accuracy(row['line'], book_sentence["text"])
        with open(output_file, 'a') as f:
            f.write(json.dumps({
                "line": row['line'],
                "book_sentence": book_sentence["text"],
                "n_gram_accuracy": n_gram_accuracy,
                "line_idx": row['idx'],
                "book_idx": j
            }) + "\n")

100%|██████████| 2000/2000 [01:30<00:00, 22.17it/s]


In [142]:
df_temp = pd.read_json(output_file, lines=True)
df_temp.sort_values(by='n_gram_accuracy', ascending=False)

,line,book_sentence,n_gram_accuracy,line_idx,book_idx
0,"` ` you are threatening life , especially to d...","usually , he would be tearing around the livin...",0.0,48848192,0
1,"` ` you are threatening life , especially to d...",but just one look at a minion sent him practic...,0.0,48848192,1
2,"` ` you are threatening life , especially to d...",that had been megan 's plan when she got him d...,0.0,48848192,2
3,"` ` you are threatening life , especially to d...","he 'd seen the movie almost by mistake , consi...",0.0,48848192,3
4,"` ` you are threatening life , especially to d...",she liked to think being surrounded by adults ...,0.0,48848192,4
...,...,...,...,...,...
4080685,he smiled and stared at the swank which awaite...,her fingers itched to run through the dark str...,0.0,68518599,995
4080686,he smiled and stared at the swank which awaite...,her mind ran straight to an illicit image of h...,0.0,68518599,996
4080687,he smiled and stared at the swank which awaite...,she cleared her throat that had run dry .,0.0,68518599,997
4080688,he smiled and stared at the swank which awaite...,"`` no , of course not . ''",0.0,68518599,998
